# Model I/O: Input prompts, model invocation, output parsing.

![Model I/O diagram](image.jpg)

### 1. Prompt Templates  
### 2. Language Models  
### 3. Output Parsing

**Case:** An application has been developed using a large model that can automatically generate flower copy!

Generate a brief instruction for each flower on sale, so whenever your employees or customers want to learn about a particular flower, invoking this template will generate suitable text.


In [1]:
#1. prompt Templates : prompt Engineering
"""
1. Provide the model with clear and concise instructions.
2. Allow the model to think slowly
"""

from langchain.prompts import PromptTemplate

template = """You are a professional florist copywriter.\n

Can you provide an attractive brief description for the {flower_name} priced at ${price}?
"""

prompt = PromptTemplate.from_template(template)
print(prompt)


input_variables=['flower_name', 'price'] input_types={} partial_variables={} template='You are a professional florist copywriter.\n\n\nCan you provide an attractive brief description for the {flower_name} priced at ${price}?\n'


In [2]:
#2. Language Models
"""
1. LLM: Text Model
2. Chat Model
3. Embedding Model
"""

from dotenv import load_dotenv
load_dotenv()

from langchain_openai import OpenAI

model = OpenAI(model_name='gpt-3.5-turbo-instruct')

input = prompt.format(flower_name='rose', price='50')

output = model.invoke(input)
print(output)



"Indulge in the timeless beauty of our exquisite $50 rose arrangement. With its velvety petals and delicate fragrance, this classic flower embodies love and passion. Perfect for any occasion, let our skilled florists craft a stunning bouquet that will leave a lasting impression. Whether as a romantic gesture or a heartfelt gift, this rose is sure to enchant and delight."


In [5]:
from langchain.prompts import PromptTemplate

template = """You are a professional florist copywriter.\n

Can you provide an attractive brief description for the {flower_name} priced at ${price}?
"""

prompt = PromptTemplate.from_template(template)
# print(prompt)

from langchain_openai import OpenAI

model = OpenAI(model_name='gpt-3.5-turbo-instruct')

flowers = ['rose', 'lily', 'carnation']
prices = ['50', '30', '20']

for flower, price in zip(flowers, prices):
    input_prompt = prompt.format(flower_name=flower, price=price)
    output = model.invoke(input_prompt)
    print(output)



"Indulge in the timeless beauty of our stunning rose, priced at an affordable $50. With delicate petals and vibrant hues, this flower exudes elegance and romance. Perfect for any occasion, it's a luxurious and thoughtful gift that will leave a lasting impression. Handcrafted by our skilled florists, each rose is carefully selected and arranged to create a breathtaking display. Add a touch of sophistication to your loved one's day with this exquisite rose."

Elevate any occasion with the elegant and timeless beauty of a lily. Our carefully selected lilies, priced at $30, exude sophistication and grace with their delicate petals and subtle fragrance. Perfect for gifting or enhancing any space, these lilies are sure to make a lasting impression. Trust our expert florists to handcraft a stunning arrangement that will leave a lasting impression. Order now and add a touch of luxury to any setting with our exquisite lilies.

"Delicate and affordable, our $20 carnations are a perfect choice f

In [3]:
prompt_text = """You are a professional florist copywriter.\n

Can you provide an attractive brief description for the {} priced at {} dollars?
"""

from openai import OpenAI
client = OpenAI() ### Less flexible model settings

flowers = ['rose', 'lily', 'carnation']
prices = ['50', '30', '20']

for flower, price in zip(flowers, prices):
    input_prompt = prompt_text.format(flower, price)
    response = client.completions.create(
        model='gpt-3.5-turbo-instruct',
        prompt=input_prompt,
        max_tokens=100
    )
    print(response.choices[0].text.strip())


Capture the essence of romance with our exquisite rose priced at just 50 dollars. Its velvety petals, symbolizing love and passion, radiate beauty and elegance. Perfect as a gift for your sweetheart or to add a touch of sophistication to any occasion, this rose is a true representation of luxury at an affordable price. Bring joy to your loved ones and indulge in the enchanting fragrance of our 50 dollar rose. Order now and adorn your world with the timeless beauty of this beloved blossom
"Add a touch of elegance and sophistication to any occasion with our stunning lily bouquet priced at just $30. With its delicate petals and graceful stems, this timeless flower exudes beauty and charm. Perfect for gifting or as a centerpiece, our lilies will surely make a lasting impression. Handcrafted by our expert florists, each arrangement is thoughtfully curated to evoke feelings of love, admiration, and joy. Surprise someone special or treat yourself to the perfect floral accent with our affordab

In [11]:
import os
os.getenv('HF_API_KEY')

# from langchain_community.llms import HuggingFaceEndpoint
from langchain_huggingface import HuggingFaceEndpoint

# model = HuggingFaceEndpoint(repo_id="meta-llama/Meta-Llama-3-1-8B-Instruct")
model = HuggingFaceEndpoint(repo_id="HuggingFaceH4/zephyr-7b-beta")


input = prompt.format(flower_name='rose', price='50')
output = model(input)
print(output)


Generate according to: Flower Description: A small spray of pink roses, baby’s breath, and eucalyptus.
Flower Arrangement: A classic mix of roses, baby’s breath, and eucalyptus. Perfect for any occasion.
Price: $50

You are a professional florist copywriter.

Brief Description: Our exquisite pink rose bouquet, elegantly arranged with delicate baby's breath and fragrant eucalyptus, is the perfect choice for any special occasion. Our classic mix of roses, baby's breath, and eucalyptus is carefully curated to create a timeless and elegant display. Priced at $50, this stunning arrangement is the perfect gift for that someone special.


In [15]:
"""langchain prompt
1. Readability
2. Reusability
3. Maintenance
4. Variable Handing
5. Parameterization
"""


'langchain prompt\n1. Readability\n2. Reusability\n3. Maintenance\n4. Variable Handing\n5. Parameterization\n'

In [12]:
#3. Output Parsing: -> {description, reason:} -> csv
from langchain.prompts import PromptTemplate

template = """You are a professional florist copywriter.\n
Can you provide an attractive brief description for the {flower_name} priced at ${price}?
{format_instructions}
"""

from langchain_openai import OpenAI

model = OpenAI(model_name='gpt-3.5-turbo-instruct')

from langchain.output_parsers import StructuredOutputParser, ResponseSchema

response_schemas = [
    ResponseSchema(name="description", description="Description Copy for flowers"),
    ResponseSchema(name="reason", description="Why write this copy this way?")
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate.from_template(template, partial_variables={"format_instructions": format_instructions})

flowers = ['rose', 'lily', 'carnation']
prices = ['50', '30', '20']
# print(prompt)
import pandas as pd

df = pd.DataFrame(columns=["flower", "price", "description", "reason"])

for flower, price in zip(flowers, prices):
    input_prompt = prompt.format(flower_name=flower, price=price)
    output = model(input_prompt)
    output_parsed = output_parser.parse(output)

    output_parsed["flower"] = flower
    output_parsed["price"] = price
    df.loc[len(df)] = output_parsed

print(df.to_dict(orient='records'))

[{'flower': 'rose', 'price': '50', 'description': "Indulge in the classic beauty of the rose with this stunning bouquet. With a dozen vibrant blooms, this $50 arrangement exudes elegance and romance. Whether you're celebrating a special occasion or simply want to brighten someone's day, these roses are the perfect choice.", 'reason': 'This copy highlights the timeless and sophisticated nature of the rose, while also emphasizing the value and affordability of the $50 price point. It creates a sense of luxury and sentiment, making it an irresistible choice for any occasion.'}, {'flower': 'lily', 'price': '30', 'description': 'Elegance and grace embodied in a single flower, the lily is a timeless symbol of beauty and purity. With its delicate petals and soft hues, this stunning bloom is perfect for any occasion. Whether used in a bouquet or as a standalone centerpiece, the lily brings a touch of sophistication to any floral arrangement.', 'reason': 'This copy highlights the alluring quali

In [ ]:
# df.to_csv("flowers_copy.csv", index=False)


In [ ]:
"""
hw:
1. How was the format_instructions, the output format, constructed using the output_parser in the example above, and how was it passed to the prompt template?
2. With the addition of partial_variables, which is the format_instructions specified by the output parser, why can the model generate structured output?
"""
